## Our highest scoring private solution

The competition is over. And I am sure that most of the teams have not choosen their best private solutions. Unfortunately .... 

But we would like to share our best private scoring solution.

Training setup:
- model - EfficentNet 6 pretrained on Noisy student
- resolution - 384x384
- Augmentations :
```python
albu.Compose([
    albu.HorizontalFlip(p=0.5),
    albu.VerticalFlip(p=0.5),

    AdvancedHairAugmentation(p=0.5, hairs_folder='/ssd_data/melanoma_classification/melanoma_hairs/'),

    albu.ShiftScaleRotate(scale_limit=0.3, rotate_limit=0, shift_limit=0.3, p=0.6, border_mode=0),

    albu.OneOf(
        [
            albu.RandomContrast(p=1),
            albu.HueSaturationValue(p=1),
        ],
        p=0.9,
    ),
    albu.RandomRotate90(),
    albu.Cutout(
        max_h_size=16,
        max_w_size=16
    ),
    Microscope(p=0.5),
])
```
- used external data from ISIC 2018 comp
- batch size 16 with 4 steps of gradient accumulation
- using MultiScaleDropout for classifier (https://arxiv.org/pdf/1905.09788.pdf)
- different learning rates (backbone - 0.0001 and classifier - 0.005)
- scheduler:
```python
torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=2,
    factor=0.25, 
    min_lr=1e-6,
    mode='max'
)
```
- Adam optimizer
- Simple BCE loss

Inference setup:
- While training 3 best checkpoints by validation roc_auc for each fold were saved
- Then for each fold SWA of these 3 checkpoints were used
- TTAs (12 TTAs): 
```python 
aug_combinations = [
    lambda : None,
    lambda : albu.HorizontalFlip(always_apply=True),
    lambda : albu.VerticalFlip(always_apply=True),
    lambda : Microscope(always_apply=True),
    lambda : albu.Compose([
        albu.VerticalFlip(always_apply=True),
        albu.HorizontalFlip(always_apply=True),
    ]),
    lambda : albu.RandomRotate90(always_apply=True)
] + [
    lambda : albu.Compose([
                            albu.HorizontalFlip(p=0.5),
                            albu.VerticalFlip(p=0.5),
        
                            AdvancedHairAugmentation(p=0.5, hairs_folder='/ssd_data/melanoma_classification/hairs/'),
    
                            albu.JpegCompression(),
                            albu.RGBShift(),
                            albu.CLAHE(),
                            albu.Blur(),

                            albu.ShiftScaleRotate(scale_limit=0.3, rotate_limit=0, shift_limit=0.3, p=0.75, border_mode=0),

                            albu.OneOf(
                                [
                                    albu.RandomContrast(p=1),
                                    albu.HueSaturationValue(p=1),
                                ],
                                p=0.9,
                            ),
                            albu.RandomRotate90(),
                            albu.Cutout(
                                num_holes=3,
                                max_h_size=16,
                                max_w_size=16
                            ),
                            Microscope(p=0.5),
                        ])] * 6
```
- And then simple average results from 5 folds 

Validation results:
- Mean CV score : 0.9308377465318983
- Std CV score : 0.004282178007858335
- OOF score : 0.92830

In [ ]:
import pandas as pd

sub = pd.read_csv('../input/best-private-sub/effnet6_image_chrisgroupkfold_fastplatoschedule_cutoutshiftsrotates_customaugs_denseelu_mdropout_difflrs_distributed_bigbatch_nodupls_384res_extrenaldata1817_batchacum4_SWA_6ttamicroscope6randomttas_meanfolds.csv')
sub.to_csv('submission.csv', index=False)

Many thanks to Kaggle team, all participants and especially my teammates:
- Rohit Agarwal (https://www.kaggle.com/rohitagarwal)
- Ashish Gupta (https://www.kaggle.com/roydatascience)
- Mukharbek Organokov (https://www.kaggle.com/muhakabartay)

Good Kaggling!